# DEMO de Elaboración y Ejecución de Benchmark

In [ ]:
# Poned vuestra API KEY de groq aquí:
groq_api_key = "<GROQ_API_KEY>"

###   Clase Benchmark + Groq

In [ ]:
!pip install groq tabulate
import json, re, requests
from groq import Groq
from tabulate import tabulate

class LLMBenchmark:
    def __init__(self, groq_api_key, models = ["llama-3.2-90b-vision-preview", "llama-3.3-70b-versatile", "mixtral-8x7b-32768", "gemma2-9b-it", "llama-3.2-3b-preview"]):
        self.groq_api_key = groq_api_key
        self.groq_client = Groq(api_key=groq_api_key)
        self.questions = []
        self.models = models

    def add_question(self, question, options, correct_answer):
        # Formateamos las opciones con letras
        formatted_options = {chr(65 + i): option for i, option in enumerate(options)}
        formatted_options_inverse = {option : chr(65 + i) for i, option in enumerate(options)}
        self.questions.append({
            "question": question,
            "options": formatted_options,
            "correct_answer": formatted_options_inverse[correct_answer] if correct_answer in formatted_options_inverse else correct_answer
        })

    def print_groq_models(self):
        url = "https://api.groq.com/openai/v1/models"
        headers = {
            "Authorization": f"Bearer {self.groq_api_key}",
            "Content-Type": "application/json"
        }
        response = requests.get(url, headers=headers)
        print(response.json())

    def add_model(self, *model_names):
        self.models.extend(model_names)

    def print_debug(self, message, debug):
        if debug:
            print(message)

    def run_benchmark(self, debug=False):
        results = {}
        for model_name in self.models:
            correct_predictions = 0
            for question_data in self.questions:
                question = question_data['question']
                options = question_data['options']
                correct_answer = question_data['correct_answer']

                # Construimos el prompt con las opciones
                options_text = "\n".join([f"{label}: {option}" for label, option in options.items()])
                prompt = f"{question}\nOpciones:\n{options_text}\n\nPor favor, elige la letra de la opción correcta y responde solo con esa letra y nada más."

                try:
                    chat_completion = self.groq_client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt}],
                        model=model_name,
                        max_tokens=50,
                        temperature=0,
                    )
                    model_answer = chat_completion.choices[0].message.content.strip().upper()

                    # Validar la respuesta con regex
                    if correct_answer.upper() in model_answer:
                        correct_predictions += 1
                        self.print_debug(f"✅ Model: {model_name} | Question: {question} | Answer: {model_answer} | Correct: {correct_answer}", debug)
                    else:
                        self.print_debug(f"❌ Model: {model_name} | Question: {question} | Answer: {model_answer} | Correct: {correct_answer}", debug)
                except Exception as e:
                    print(f"Error querying {model_name}: {e}")

            # Calcula la tasa de aciertos
            total_questions = len(self.questions)
            accuracy = correct_predictions / total_questions if total_questions else 0
            results[model_name] = (correct_predictions, total_questions, accuracy)

        # Imprimir resultados en una tabla
        headers = ["Modelo", "Respuestas correctas", "Exactitud"]
        table = [[model, f"{correct}/{total}", f"{accuracy:.2%}"] for model, (correct, total, accuracy) in results.items()]
        print(tabulate(table, headers=headers, tablefmt='grid'))
        #return results

###   Ejemplos

In [ ]:
llm_benchmark_test = LLMBenchmark(groq_api_key)
llm_benchmark_test.add_question("¿Cuál es la capital de Francia?", ["París", "Roma", "Berlín"], "A")
llm_benchmark_test.add_question("¿Cuál es el río más largo del mundo?", ["Nilo", "Amazonas", "Danubio"], "Nilo")
llm_benchmark_test.add_question("¿En qué año se descubrió América?", ["1492", "1776", "1642"], "1492")
llm_benchmark_test.add_question("¿Cuál es la fórmula química del agua?", ["H2O", "CO2", "NaCl"], "A")
llm_benchmark_test.run_benchmark(True)

In [ ]:
llm_benchmark_test2 = LLMBenchmark(groq_api_key)
llm_benchmark_test2.add_question("¿Cuál es el elemento químico más abundante en la corteza terrestre?", ["Oxígeno", "Silicio", "Aluminio"], "Oxígeno")
llm_benchmark_test2.run_benchmark(True)

#   CÓDIGO PRÁCTICA

In [ ]:
llm_benchmark = LLMBenchmark(groq_api_key)
# A completar